In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Начало работы
## Нужно найти очистить данные и найти признаки, которые являются наиболее важными факторами, влияющие на удовлетворенность авиапассажиров

## Импортируем данные
### Первым делом нам необходимо импортировать данные из системы kaggle. У нас должно получиться два датафрейма:
1. ### датафрейм для тренировки модели
2. ### датафрейм для тестирования модели, итоговый датасет

### На будущее, нам необходимо объединить датасеты в один, осуществить все манипуляции с данными и потом разделить

In [ ]:
df_train = pd.read_csv('/kaggle/input/airline-passenger-satisfaction/train.csv')
df_test = pd.read_csv('/kaggle/input/airline-passenger-satisfaction/test.csv')

In [ ]:
length = len(df_train)

df_all = pd.concat([df_train, df_test])

### После импорта данных мы можем посмотреть краткую информацию о датасетах, что мы загрузили - команда .head подойдет в самый раз (первые пять строк)

In [ ]:
df_all.head()

### Можно заметить следующее:
1. ### у нас есть лишний столбец, который будет нам мешать. В последующем его необходимо будет удалить
2. ### признаки в тренировочном и итоговом датасете идентичные, каждый признак из тренировочного датасета присутствует в итоговом
### Далее нужно провести осмотр всех типов данных в датасетах - команда .info 

In [ ]:
df_all.info()

### После исполнения команды мы заметили:
* ### присутствует три типа данных: int64, object и float64 (float64 - один признак, object - пять признаков, остальные int64)
* ### все признаки заполнены данными кроме одного - Arrival Delay in Minutes (опоздание рейса в минутах). Данный признак имеет пустые значения, поэтому нужно выяснить, какими данными нам заполнить эти пропуски

## Заполняем пропуски в данных

In [ ]:
import matplotlib.pyplot as plt
#df['Arrival Delay in Minutes'].value_counts(dropna=False, ascending = True)
df_all['Arrival Delay in Minutes'].hist(bins = 100, range = (0,200))
plt.show()

In [ ]:
df_all['Arrival Delay in Minutes'].describe()

### Из графика выше и команды .describe строится мысль, что если мы используем медиану, то у нас выйдет от 0 до 1, ведь 0-ей гораздо больше (они занимают больше 50% всех данных). А если возьмем среднее, то у нас будет 15 минут, а это уже будет неверным способом заполнения данных, ведь задержка рейса на 1584 минуты это редкость, выброс, который лучше не использовать в данных. 
### Резюмируя выше сказанное, выбираем медиану в качестве заполнителя - используем команду .fillna для заполнения всех пропусков

In [ ]:
medianArrivalDelay = df_all['Arrival Delay in Minutes'].median()
df_all['Arrival Delay in Minutes'] = df_all['Arrival Delay in Minutes'].fillna(medianArrivalDelay)
df_all['Arrival Delay in Minutes'].isna().sum()

## Преобразование категорийных данных в количественные

### После заполнения пропусков в данных признака Arrival Delay in Munutes нам нужно заменить категорийные данные, ведь модель не сможет функционировать с данными типа Object. Однако в первую очередь, нужно выяснить, какие данные являются категорийными

In [ ]:
df_all.info()

In [ ]:
objectColumn = ['Gender', 'Customer Type', 'Type of Travel', 'Class', 'satisfaction']
for column in objectColumn:
    print(df_all[column].value_counts().count())

### Как мы ранее отметили, в нашем датасете есть пять признаков с типом Object. Далее мы проверили, а сколько уникальных значений есть в каждом из этих признаков. Как оказалось, все признаки категорийные, то есть их можно разделить на классы
### Теперь самое интересное, как мы их преобразуем в числа?? А здесь нам поможет one hot encoding с применением команды .get_dumnies (данная команда осуществляет преобразование категориальной переменной в фиктивные / индикаторные переменные)
### Для создания функционала, мы напишем код функции, которая принимает в себя два значения: датасет и столбец. Преобразованный столбец сохраняем в отдельной переменной, а потом подставляем его в наш изначальный датасет. Последними действия делаем удаление столбца со старыми значениями и возвращаем наш обновленный датасет. Все это делается в цикле, куда мы заносим все столбца с категориальным типом

In [ ]:
def onehot_encode(df, column):
    df = df.copy()
    dummies = pd.get_dummies(df[column], prefix=column, drop_first = True)
    df = pd.concat([df, dummies], axis=1)
    df = df.drop(column, axis=1)
    return df

In [ ]:
for column in objectColumn:
    df_all = onehot_encode(df_all, column)

In [ ]:
df_all.head()

### Как видно, наша таблица обновилась, те признаки, что были категориальными со строчными данными стали числовыми и добавились в конец датасета

## Тренировка модели

### Для тренировки модели нам необходимо разделить общую таблицу на два датасета: тренировочный и тестовый. Далее необходимо будет раззделить выборки на данные, по которым мы будем предсказывать и данные, которые мы будем предсказывать

In [ ]:
df_all.head()

In [ ]:
df_train = df_all[:length]
df_test = df_all[length:]


X = df_train[df_train.columns[:-1]]
X_test = df_test[df_test.columns[:-1]]

y = df_train['satisfaction_satisfied']
y_test = df_test['satisfaction_satisfied']

### После разделения данных, мы их делим тренировочные датасеты и валидационные, на которых мы будем проверять 

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 2020)

### В качестве моделей предсказания, мы выбрали линейную регрессию и дерево решений, в качестве метрики качества - mae

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error, accuracy_score

modelLogReg = LogisticRegression()
modelLogReg.fit(X_train, y_train)

scorePredicted = modelLogReg.predict(X_val)
#scorePredictedSeries = pd.Series()

result = accuracy_score(y_val ,scorePredicted)
result

In [ ]:
modelDecTree = DecisionTreeClassifier()
modelForest =  RandomForestClassifier(n_estimators = 1000)
# XGBClassifier()
models = [modelDecTree, modelForest]
for model in models:
    model.fit(X_train, y_train)
    
    scorePredicted = model.predict(X_val)
    result = accuracy_score(y_val ,scorePredicted)
    print(result)

### В итоге получается, что линейная регрессия показала себя лучше. Именно ее мы будем использовать, для поиска признаков, которые влияют в большей степени на удовлетворенность пассажиров
### Поиск признаков будет строиться по команде .coef_модели. Далее через цикл мы узнаем самые важные признаки для предсказания удовлетворенности пассажиров

In [ ]:
importance = modelForest.feature_importances_

for i, coef in enumerate(importance):
    if abs(coef) > 0.05:
        print(X_train.columns[i], coef)

In [ ]:
importance

## Результат
### В итоге мы видим, что с нашим циклом что-то пошло не так. Однако мы все равно можем посмотреть на то, какие признаки в большей степени влияют на качество полетов. Эти признаки: 
* ### unnamed (наш неизвестный признак)
* ### age (возраст пассажира)
* ### Inflight wifi service (Вай-фай на борту)
* ### Online Boarding (Онлайн услуги компании)
* ### Seat comfort (удобные места)
* ### Gender_male (пол)
### Именно на эти признаки стоит обращать внимание авиакомпаниям, которые хотят повысить комфорт для своих клиентов

In [ ]:
y_val

In [ ]:
scorePredicted

In [ ]:
y_train